In [59]:
import pickle

with open('细胞系.pkl', 'rb') as file:
    cell_dict_W = pickle.load(file)
    cell_dict_columns = pickle.load(file)
    cell_dict_high_lines = pickle.load(file)

In [60]:
with open('药物对.pkl', 'rb') as file:
    drug_dict_W = pickle.load(file)
    drug_dict_columns = pickle.load(file)
    drug_dict_high_lines = pickle.load(file)

In [61]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
#from bokeh.plotting import figure, show, output_file


data1 = pd.read_excel(r'pcbi.1006752.s001.xlsx', sheet_name="Validated targets",index_col='Unnamed: 0')
data2 = pd.read_excel(r'Label_synergy4000.xlsx')
data2_columns = data2.columns[2:]

In [62]:
data2

,DrugA,DrugB,A2058_SKIN,A2780_OVARY,A375_SKIN,A427_LUNG,CAOV3_OVARY,COLO320_LARGE_INTESTINE,EFM192A_BREAST,ES2_OVARY,...,SKMES1_LUNG,SKOV3_OVARY,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE,ZR751_BREAST,DLD1_LARGE_INTESTINE,PA1_OVARY
0,MK-5108,SORAFENIB,-9.51,2.60,15.20,6.22,-16.4000,5.360,-7.080,8.050,...,-6.230,13.50,5.10,5.15,6.63,3.880,-1.48,-8.660,13.00,4.83
1,VINORELBINE,SUNITINIB,-13.20,-4.03,11.10,10.50,-15.6000,16.300,-1.720,8.890,...,2.820,1.51,13.80,-5.77,-14.40,-10.700,-16.80,-11.800,15.30,-13.10
2,SUNITINIB,MK-8776,26.40,14.50,29.50,17.50,17.7000,10.300,0.830,5.770,...,-12.200,-7.40,10.50,-4.97,13.10,18.800,12.00,5.950,6.77,18.90
3,5-FU,DINACICLIB,4.33,-8.16,-5.42,-7.75,-14.3000,4.810,0.312,3.410,...,-3.140,-13.90,-4.57,-19.60,-2.72,3.680,14.40,-13.600,-3.64,-5.73
4,SUNITINIB,MK-2206,45.50,22.50,22.80,18.00,20.1000,22.700,6.300,24.900,...,24.400,31.20,18.40,12.90,14.30,34.700,21.20,14.400,26.40,55.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,CARBOPLATIN,MK-2206,1.35,11.70,20.20,9.17,46.6000,0.586,-37.700,-2.100,...,-0.217,-10.90,22.00,-2.43,-43.90,9.340,-4.83,-15.400,-4.42,17.80
579,5-FU,LAPATINIB,9.19,8.01,7.50,-4.18,-6.7800,3.210,7.240,5.340,...,11.100,7.12,-2.42,6.96,14.90,9.120,10.60,14.700,11.60,14.20
580,ETOPOSIDE,AZD1775,15.40,12.40,19.20,-5.01,37.4000,8.590,9.770,3.770,...,4.330,5.84,-1.58,18.60,-20.70,25.100,20.30,3.750,-4.29,-1.44
581,BORTEZOMIB,MK-8669,-6.30,4.79,-23.30,-2.70,0.0856,-17.300,19.200,17.400,...,-13.900,-2.42,-2.61,19.10,-11.10,-13.500,9.05,-0.306,9.48,-4.29


In [63]:
# 结构  {名字： 名字列表， 权重： 权重列表} 顺序对应
dict_molecule_cell = {}
for elem in cell_dict_high_lines:
    dict_molecule_cell[elem] = []
    dict_molecule_cell[elem].append(cell_dict_columns[elem]["max_relevance"][-5:])
    dict_molecule_cell[elem].append(cell_dict_W[elem][-5:])

In [64]:
dict_molecule_drug = {}
for elem in drug_dict_high_lines:
    dict_molecule_drug[elem] = []
    dict_molecule_drug[elem].append(drug_dict_columns[elem]["max_relevance"][-5:])
    dict_molecule_drug[elem].append(drug_dict_W[elem][-5:])

In [65]:
# molecule = 0
# denominator = 0
# for i,j in zip(dict_molecule_cell, dict_molecule_drug):
#     a = dict_molecule_cell[i][0]
#     b = dict_molecule_drug[j][0]
#     cw = dict_molecule_cell[i][1]
#     dw = dict_molecule_drug[j][1]
#     for m,ccw in zip(a,cw): # 细胞系   名字  权重
#         for n,ddw in zip(b,dw): # 药物对   名字   权重
#             # 分子子项 = 细胞系权重 × data2对应名字的值 ×药物对权重
#             temp_molecule = ccw*data2[(data2["DrugA"]==n.split(", ")[0][1:-1]) & (data2["DrugB"]==n.split(", ")[1][1:-1])][m].values[0]*ddw
#             # 分母子项 = 细胞系权重 × 药物对权重
#             temp_denominator = ccw*ddw
#             molecule += temp_molecule
#             denominator += temp_denominator
# print(molecule, denominator)

In [66]:
drugA = []
drugB = []
for i in dict_molecule_drug:
    drugA.append(i.split(", ")[0][1:-1])
    drugB.append(i.split(", ")[1][1:-1])

In [67]:
temp = []
for index,row in data2.iterrows():
    status = 0
    for i,j in zip(drugA, drugB):
        if i == row[0] and j == row[1]:
            status = 1
    if status == 1:
        temp.append(row)
data22 = pd.DataFrame(temp)
data22

,DrugA,DrugB,A2058_SKIN,A2780_OVARY,A375_SKIN,A427_LUNG,CAOV3_OVARY,COLO320_LARGE_INTESTINE,EFM192A_BREAST,ES2_OVARY,...,SKMES1_LUNG,SKOV3_OVARY,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE,ZR751_BREAST,DLD1_LARGE_INTESTINE,PA1_OVARY
1,VINORELBINE,SUNITINIB,-13.200,-4.030,11.1000,10.5000,-15.600,16.300,-1.72,8.890,...,2.8200,1.510,13.800,-5.7700,-14.40,-10.700,-16.800,-11.8000,15.300,-13.100
19,AZD1775,MK-4827,32.800,9.070,20.4000,7.9800,-0.744,-1.650,8.68,-5.590,...,15.0000,17.100,4.260,17.8000,11.00,21.800,4.640,11.7000,-1.870,31.800
30,VINORELBINE,DASATINIB,0.844,40.100,32.2000,39.4000,-14.900,-0.248,16.10,19.500,...,10.2000,25.100,31.300,20.9000,45.20,-15.900,-10.400,-15.5000,19.800,-5.170
38,VINBLASTINE,MK-4827,-13.400,-11.400,-19.0000,-12.1000,-33.500,11.700,15.50,11.800,...,-7.5100,21.100,30.900,-21.7000,-48.80,-1.630,16.400,-2.0700,-3.680,-16.300
41,VINBLASTINE,AZD1775,6.790,1.420,-5.4900,-13.3000,-7.450,3.480,-10.10,21.100,...,-11.1000,25.800,20.400,-2.7000,-23.20,16.600,-10.600,-4.5300,0.656,-2.240
43,MK-4827,DASATINIB,15.100,65.100,31.8000,20.3000,5.400,15.500,33.80,-4.520,...,0.9250,-5.520,37.100,19.0000,-15.40,4.860,29.800,27.4000,4.530,26.600
79,AZD1775,TOPOTECAN,5.720,4.650,-3.3900,11.3000,3.240,3.240,-5.55,-6.470,...,-11.4000,7.650,17.100,7.6700,1.82,4.170,0.328,15.0000,2.470,3.100
90,VINORELBINE,AZD1775,-15.900,-7.690,-2.0500,-5.4400,-13.700,6.880,-3.56,4.440,...,-7.9000,15.300,-0.374,-11.2000,-13.60,-1.620,-33.100,-16.4000,4.810,-8.200
105,SUNITINIB,TOPOTECAN,-7.880,2.120,-9.1600,12.7000,2.270,5.950,2.19,-9.190,...,-5.0100,-11.500,5.240,-5.6800,-23.30,-4.970,6.640,-0.2710,-13.600,-6.620
120,VINBLASTINE,DINACICLIB,-17.100,-20.200,-3.7900,-16.6000,-32.200,0.731,-8.27,11.800,...,-27.5000,-21.100,12.700,-19.2000,-60.00,-2.470,-17.000,-30.9000,-6.890,-26.800


In [68]:
cells = []
for i in dict_molecule_cell:
    cells.append(i)
cells.extend(data22.columns[0:2])

In [69]:
temp = []
for column,line in data22.iteritems():
    if column in cells:
        temp.append(line)
data222 = pd.DataFrame(temp).T
data222.shape

(56, 13)

In [70]:
tptt = []
for i,j in zip(data222['DrugA'], data222['DrugB']):
    tptt.append(f"'{i}', '{j}'")
data222 = pd.DataFrame(data222.iloc[:,2:].values,index=tptt,columns=data222.columns[2:])

In [71]:
tptt = []
for i,j in zip(data2['DrugA'], data2['DrugB']):
    tptt.append(f"'{i}', '{j}'")
data3 = pd.DataFrame(data2.iloc[:,2:].values,index=tptt,columns=data2.columns[2:])
data2 = data3.copy()

In [72]:
# result_df = []
# for i,j in zip(data2['DrugA'], data2['DrugB']):
#     tp = []
#     for k in data2.columns[2:]:
#         molecule = 0
#         denominator = 0
#         a = dict_molecule_cell[k][0]
#         b = dict_molecule_drug[f"'{i}', '{j}'"][0]
#         cw = dict_molecule_cell[k][1]
#         dw = dict_molecule_drug[f"'{i}', '{j}'"][1]
#         for m,ccw in zip(a,cw): # 细胞系   名字  权重
#             for n,ddw in zip(b,dw): # 药物对   名字   权重
#                 # 分子子项 = 细胞系权重 × data2对应名字的值 ×药物对权重
#                 temp_molecule = ccw*data2[(data2["DrugA"]==n.split(", ")[0][1:-1]) & (data2["DrugB"]==n.split(", ")[1][1:-1])][m].values[0]*ddw
#                 # 分母子项 = 细胞系权重 × 药物对权重
#                 temp_denominator = ccw*ddw
#                 molecule += temp_molecule
#                 denominator += temp_denominator
#         result = molecule / denominator
#         tp.append(result)
#     result_df.append(tp)

In [73]:
dict_molecule_drug["'VINORELBINE', 'SUNITINIB'"]

[Index([''MK-4541', 'VINORELBINE'', ''L778123', 'VINORELBINE'',
        ''VINORELBINE', 'SORAFENIB'', ''DOXORUBICIN', 'SUNITINIB'',
        ''VINBLASTINE', 'SUNITINIB''],
       dtype='object'),
 array([0.01964879, 0.03848276, 0.04145739, 0.04145739, 0.67801402])]

In [74]:
len(dict_molecule_drug)

56

In [75]:
len(dict_molecule_cell)

11

In [76]:
data2.loc["'VINBLASTINE', 'SUNITINIB'",:]

A2058_SKIN                   1.6900
A2780_OVARY                 -3.7000
A375_SKIN                    1.1300
A427_LUNG                    4.1200
CAOV3_OVARY                 -3.7900
COLO320_LARGE_INTESTINE     18.0000
EFM192A_BREAST               7.5900
ES2_OVARY                   16.7000
HCT116_LARGE_INTESTINE       5.8200
HT144_SKIN                   7.0100
HT29_LARGE_INTESTINE       -11.5000
KPL1_BREAST                 14.7000
LOVO_LARGE_INTESTINE        -0.2710
MDAMB436_BREAST             -5.8900
MSTO211H_PLEURA            -12.2000
NCIH1650_LUNG                7.6100
NCIH2122_LUNG                4.9700
NCIH23_LUNG                -18.9000
NCIH460_LUNG                -3.9200
NCIH520_LUNG                -0.0105
OV90_OVARY                   0.0492
OVCAR4_OVARY                -0.8430
RKO_LARGE_INTESTINE         -3.0000
RPMI7951_SKIN              -18.0000
SKMEL30_SKIN               -10.3000
SKMES1_LUNG                 -9.7000
SKOV3_OVARY                  4.8200
SW620_LARGE_INTESTINE       

In [77]:
result_df = []
for i in data222.index:
    tp = []
    for k in data222.columns[2:]:
        molecule = 0
        denominator = 0
        a = dict_molecule_cell[k][0]
        b = dict_molecule_drug[i][0]
        cw = dict_molecule_cell[k][1]
        dw = dict_molecule_drug[i][1]
#         print(len(a), len(b), len(cw), len(dw))
        for m,ccw in zip(a,cw): # 细胞系   名字  权重
            for n,ddw in zip(b,dw): # 药物对   名字   权重
                # 分子子项 = 细胞系权重 × data2对应名字的值 ×药物对权重
                # print(m, ccw, n, ddw)
#                 if data2[data2.index==n][m].values == np.array(list([])):
#                     print(m, ccw, n, ddw)
                temp_molecule = ccw*data2[data2.index==n][m].values[0]*ddw
                # print([data2[data2.index==n][m].values[0]] == [])
                # 分母子项 = 细胞系权重 × 药物对权重
                temp_denominator = ccw*ddw
                molecule += temp_molecule
                denominator += temp_denominator
        result = molecule / denominator
        tp.append(result)
#     result = 0
#     print(i,k)
    result_df.append(tp)

In [78]:
pd.DataFrame(np.array(result_df))

,0,1,2,3,4,5,6,7,8
0,-0.708648,-0.626749,11.986977,11.084550,-2.755858,-3.029928,-0.192687,-2.937410,9.930374
1,15.833042,14.164620,7.921185,9.542098,9.870995,10.657731,12.423299,11.660510,9.553698
2,20.284014,29.170124,20.483904,9.359639,-8.978235,8.350068,-23.910216,15.486561,11.316348
3,-14.352622,-14.508159,-4.291346,-5.325890,-15.798262,-27.865315,-17.002803,-17.598816,-3.954208
4,6.340132,2.471515,5.225040,4.805415,-8.296774,6.983986,-10.676235,4.970495,4.275542
5,48.507171,57.189282,49.529270,44.616852,23.615397,35.924285,9.405365,41.623260,40.620298
6,9.041751,3.235054,5.236648,4.664170,-9.603309,7.932216,-16.300267,4.876591,5.683396
7,8.026663,3.589721,9.382325,8.202491,-4.211118,4.291383,-2.736163,2.218810,7.271192
8,3.430767,2.857566,4.674333,2.104696,6.283604,2.644687,1.120101,1.425984,6.706591
9,-8.447927,-13.278612,-1.248623,-3.624552,-11.926442,-12.807659,-12.326830,-10.329333,-1.616584
